# Notebook to access to data for the Science Verification Sprint of February 2025

- Confluence page : https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/372867091/System-level+Science+Performance+Verification+Sprint
- slides : https://docs.google.com/presentation/d/1bPjS5NGtiEL2lfjmvP1UYdG_tMEDfZxX26ayhf7MhtY/edit#slide=id.ga2f7fb1a1f_0_70

- where to find the campains
- https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

- Notebooks examples
- https://github.com/lsst-dm/DMTR-401/blob/main/notebooks/test_LVV-T40_T1240.ipynb
- https://github.com/lsst-dm/DMTR-412/blob/tickets/DM-38728/notebooks/test_LVV-T1751_AM1_AM2.ipynb

In [1]:
# Confirm that the version of the Science Pipelines is recent:
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

dagoret-nb
lsst_distrib          gc7ba34d93f+8892a0c6f9 	current w_2024_50 setup
